In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import numpy as np

import dataProcess

from datetime import datetime

In [2]:
hourly = '.\\Raw data\\ten_year\\weatherstats_toronto_hourly.csv'
daily = '.\\Raw data\\ten_year\\weatherstats_toronto_daily.csv'
start = datetime(2014, 4, 16).date()
end = datetime(2024, 4, 10).date()
data = dataProcess.generateDataNoLoader(hourly, daily, start, end)
train_data = data['train']
val_data = data['validation']
test_data = data['test']

In [21]:
class FlatWeatherMLP(nn.Module):
    def __init__(self, input_size=312, hidden_size=500):
        super(FlatWeatherMLP, self).__init__()
        self.inputLayer = nn.Linear(input_size, hidden_size)
        self.hiddenLayer = nn.Linear(hidden_size, hidden_size)
        self.outputLayer = nn.Linear(hidden_size, 1)
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = x.to(torch.float32)
        x = torch.flatten(x, start_dim=1)
        # print(x.shape)
        x = self.inputLayer(x)
        x = self.relu(x)
        # print(f'fc1: {x}')
        x = self.hiddenLayer(x)
        x = self.relu(x)
        # print(f'fc2: {x}')
        x = self.outputLayer(x)
        # print(f'fc3: {x}')
        # print(x)
        x = torch.squeeze(x)
        return x

In [25]:
# for d, target in data['train']:
#     print(d.dtype)
#     print(target.shape)
def accuracy(model, data):
    totalDiff = 0.0
    count = 0
    loader = DataLoader(data, batch_size=100)
    for data, target in loader:
        count = count + 1
        z = model(data)
        # print(z)
        totalDiff += torch.sum(torch.abs(target - z))

    print(target, z)
    return(totalDiff / count)

# accuracy(FlatWeatherMLP(), data['train'])
        


In [26]:

# torch.set_printoptions(threshold=10_000)
def train_model(model, trainData, valData, lr=.001, batchSize=10, epochs=50):
    loader = DataLoader(train_data, batch_size=batchSize, shuffle=True)
    vloader = DataLoader(val_data, batch_size=batchSize, shuffle=True)
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    model.train()
    for epoch in range(epochs):
        totalLoss = 0
        c = 0
        for d, t in loader:
            t = torch.squeeze(t)
            c += 1
            if torch.isnan(d).any():
                # torch.set_printoptions(profile="full")
                print(d)
                print(c)
            optimizer.zero_grad()
            z = model(d)
            loss = criterion(z, t)
            loss.backward()
            optimizer.step()
            totalLoss += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Training Loss: {totalLoss / len(loader)}')
        # print(z)
        # print(t)
        model.eval()
        totalLossVal = 0
        with torch.no_grad():
            for d, t in vloader:
                z = model(d)
                loss = criterion(z, t)
                totalLossVal += loss.item()
        print(f'Epoch [{epoch+1}/{epochs}], Validation Loss: {totalLossVal / len(loader)}')
    print(z)
model = FlatWeatherMLP()
train_model(model, train_data, val_data)
accuracy(model, data['validation'])



Epoch [1/50], Training Loss: 206621.48416139453
Epoch [1/50], Validation Loss: 58.75657249485842


c:\Users\Blake\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch [2/50], Training Loss: 181.87400362129037
Epoch [2/50], Validation Loss: 121.12791755781745
Epoch [3/50], Training Loss: 116.25881051797471
Epoch [3/50], Validation Loss: 39.152386341226816
Epoch [4/50], Training Loss: 86.7115776220225
Epoch [4/50], Validation Loss: 50.13700468529205
Epoch [5/50], Training Loss: 85.39751760421261
Epoch [5/50], Validation Loss: 29.109757304740942
Epoch [6/50], Training Loss: 78.77022468988797
Epoch [6/50], Validation Loss: 31.09912934720791
Epoch [7/50], Training Loss: 77.70994340659287
Epoch [7/50], Validation Loss: 34.96623302055394
Epoch [8/50], Training Loss: 66.77502550397601
Epoch [8/50], Validation Loss: 36.017357792173115
Epoch [9/50], Training Loss: 50.96859784939322
Epoch [9/50], Validation Loss: 27.321239042941325
Epoch [10/50], Training Loss: 47.28140971627653
Epoch [10/50], Validation Loss: 40.21550493416149
Epoch [11/50], Training Loss: 45.77997938498923
Epoch [11/50], Validation Loss: 58.08850899270053
Epoch [12/50], Training Loss: 

tensor(468.1586, grad_fn=<DivBackward0>)